# Лабораторная работа №1
Выполнил: Шубин Г.С. М8О-106М-23

## Загрузка датасета и предпроцессинг

In [51]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time

In [52]:
df = pd.read_csv("/kaggle/input/folder/data.csv")

In [53]:
df.head()

,Отметка времени,пол,возраст,кто ты?,во сколько встаете?,далеко ли живешь от института?,особенности характера,работаете?,доход,был на паре 07.08
0,09.09.2023 15:23:00,ж,22,сова,8.00,да,интроверт,да,доволен,да
1,09.09.2023 15:25:11,м,21,сова,9:00,Нет,интроверт,да,недоволен,Нет
2,09.09.2023 15:25:43,м,22,сова,10,да,интроверт,Нет,недоволен,да
3,09.09.2023 15:26:02,м,22,сова,Время когда лягу + 7 часов сна,да,экстраверт,да,доволен,Нет
4,09.09.2023 15:27:41,ж,21,сова,11,да,интроверт,да,доволен,да


In [54]:
df["Отметка времени"] = pd.to_datetime(df["Отметка времени"])
df["пол"] = df["пол"] == "м"
df["кто ты?"] = df["кто ты?"] == "сова"
df["во сколько встаете?"] = df["во сколько встаете?"]
df["далеко ли живешь от института?"] = df["далеко ли живешь от института?"].str.strip().str.lower() == "да"
df["особенности характера"] = df["особенности характера"] == "интроверт"
df["работаете?"] = df["работаете?"].str.strip().str.lower() == "да"
df["доход"] = df["доход"] == "доволен"
df["был на паре 07.08"] = df["был на паре 07.08"].str.strip().str.lower() == "да"

def get_time_wakeup(obj: str):
    obj = obj.replace(".", ":")
    try:
        wakeup = time.strptime(obj, "%H:%M")
        return wakeup.tm_hour*60+wakeup.tm_min
    except ValueError:
        pass
    try:
        wakeup = time.strptime(obj, "%H")
        return wakeup.tm_hour*60+wakeup.tm_min
    except ValueError:
        return None
    return obj

df["во сколько встаете?"] = df["во сколько встаете?"].map(get_time_wakeup)

In [55]:
df = df.dropna()

In [56]:
X = df.drop(["был на паре 07.08", "Отметка времени"], axis=1)
y = df["был на паре 07.08"]

# Обучение модели

In [57]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=40)
X_train.shape, y_train.shape, X_val.shape, y_val.shape

((8, 8), (8,), (3, 8), (3,))

## Алгоритм kNN

In [58]:
from sklearn.metrics import euclidean_distances
from sklearn.base import BaseEstimator, ClassifierMixin

class MyKNN(BaseEstimator, ClassifierMixin):
    def __init__(self, n_neighbors=5):
        self.n_neighbors = n_neighbors
        
    def fit(self, X, y):
        self.fit_X = X.values.tolist()
        self.fit_y = y.values.tolist()
        return self
    
    def predict(self, X):
        distances = np.zeros(X.shape[0])  # Initialize distances as an array of zeros
        
        if len(X.shape) == 1:
            distances = euclidean_distances(X.values.tolist(), self.fit_X)[0]
        else:
            distances = euclidean_distances(X.values.tolist(), self.fit_X)
        distances_sorted = np.argsort(distances, axis=1)
        top_n = distances_sorted[:, :self.n_neighbors]
        
        answer = np.empty(X.shape[0])
        for i, elem in enumerate(top_n):
            local_ans = dict()
            for num in elem:
                if self.fit_y[num] in local_ans.keys():
                    local_ans[self.fit_y[num]] += 1
                else:
                    local_ans[self.fit_y[num]] = 1
            max_count = max(local_ans, key=lambda k: local_ans.get(k))
            answer[i] = max_count

        return answer

In [59]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

clf = KNeighborsClassifier(n_neighbors=5)
my_clf = MyKNN()
clf.fit(X_train, y_train)
my_clf.fit(X_train, y_train)

MyKNN()

## Сравнение точности моделей

In [60]:
pred = clf.predict(X_val)
accuracy_score(y_val, pred)

0.6666666666666666

In [61]:
my_pred = my_clf.predict(X_val)
accuracy_score(y_val, my_pred)

0.6666666666666666